In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codeseys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codeseys/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/codeseys/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codeseys/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df_train = pd.read_csv('training_data.csv')
df_train.head()

,ID,User,Text,Sentiment
0,864192,Carly_FTS,I *heart* filling up @dennisschaub desk 1 it...,1
1,523691,Open_Sourcing,"#SocioMat - people create prettier, younger an...",1
2,584154,xxcharlx,no way i dont want the tour to end,0
3,1527961,andreapuddu,@HemalRadia Hi Amazing Brother! Sending Limitl...,1
4,28609,umbec,@flockmaster they are chocolate,1


In [3]:
stopwordslist =  set(stopwords.words('english'))
lemmatty = WordNetLemmatizer()
wordnetmap = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def removestopwords(text):    
    return " ".join([word for word in str(text).split() if word not in stopwordslist])
def lemmy(text):
    pos_text = nltk.pos_tag(str(text).split())
    return " ".join([lemmatty.lemmatize(word,wordnetmap.get(pos[0],wordnet.NOUN))for word, pos in pos_text])    

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'#','',text)
    
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    
    text = lemmy(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]','', text)
    text = removestopwords(text)
    
    return text

# df_train['Text']=df_train['Text'].apply(clean_text)
# df_train.head()

In [4]:
TextBlob(df_train['Text'][1]).words

WordList(['SocioMat', 'people', 'create', 'prettier', 'younger', 'and', 'better', 'looking', 'avatars', 'of', 'themselves'])

In [5]:
# df_train.to_csv('newcsvtest.csv')
SEED=114
x = df_train['Text']
y = df_train['Sentiment']

x_train, x_validation_test, y_train, y_validation_test = train_test_split(x,y,test_size=0.2,random_state=SEED)

x_validation,x_test,y_validation,y_test=train_test_split(x_validation_test,y_validation_test,random_state=SEED)

print("xTrain:",x_train.shape,"yTrain:",y_train.shape)
print("xTest:",x_test.shape,"yTest:",y_test.shape)
print("xValidation:",x_validation.shape,"yValidation:",y_validation.shape)

xTrain: (800000,) yTrain: (800000,)
xTest: (50000,) yTest: (50000,)
xValidation: (150000,) yValidation: (150000,)


In [6]:
tfidf = TfidfVectorizer(analyzer=clean_text)
tfidfvect = tfidf.fit_transform(x_test)
tfidfvect.shape

(50000, 317)

In [10]:
ndf = pd.DataFrame(tfidfvect.toarray(),columns=tfidf.get_feature_names())
ndf.head()

,,0,1,2,3,4,5,6,7,8,...,참,철,킨,테,표,하,한,해,했,회
0,0.579624,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.486722,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.351588,0.0,0.35667,0.341409,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.360189,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.370298,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
